# Write article ids to a file

In [1]:
import xml.etree.ElementTree as ET
import urllib2
retmax = 1000 
ids_page_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=bcl2&retmax=" + str(retmax)
ids_xml_page = urllib2.urlopen(ids_page_url).read()

ids_root = ET.fromstring(ids_xml_page)
# write ids to article-ids.txt
with open('raw_data/article-ids.txt', 'w+') as my_file:
    for ids in ids_root.iter('IdList'):
        for id in ids.iter('Id'):
            my_file.write(id.text.encode('utf-8') + '\n')

# Get article urls (non-spark method)

In [ ]:
from bs4 import BeautifulSoup

abstract_url_base = "https://www.ncbi.nlm.nih.gov/pubmed/"
# write urls to article-urls.txt
with open('raw_data/article-urls.txt', 'w+') as my_file:
    for ids in ids_root.iter('IdList'):
        for id in ids.iter('Id'):
            # build the full pubmed url
            url = abstract_url_base + id.text.encode('utf-8')
            html_content = urllib2.urlopen(url)
            if(html_content):
                soup = BeautifulSoup(html_content)
                full_text_link = soup.find('a', attrs={'free_status': 'free'})
                if(full_text_link):
                    my_file.write(id.text.encode('utf-8') + '\t' + full_text_link.attrs['href'] + '\n')

In [3]:
#BeautifulSoup(urllib2.urlopen('https://www.ncbi.nlm.nih.gov/pubmed/28391633'))

## Load article url data

In [4]:
article_urls = spark.read.csv('raw_data/article-urls.txt', sep='\t', inferSchema=True).toDF('pubmed_id', 'url')

NameError: name 'spark' is not defined

In [ ]:
article_urls.show(5, truncate=False)

In [398]:
def get_article_content(url):
    req = urllib2.Request(url, headers={'User-Agent' : "Magic Browser"})
    try:
        html_content = urllib2.urlopen(req)
    except:
        pass
    return(True)
#     if(html_content):
#         soup = BeautifulSoup(html_content)
#         article = ''
#         for p in soup.find_all('p'):
#             article = article + p.text
#         article = article.replace('\r', ' ').replace('\n', ' ')
#         return(article)
get_article_content_udf = udf(get_article_content, StringType())

In [399]:
articles = article_urls.select(article_urls.url, get_article_content_udf(article_urls.url).alias('article'))

In [ ]:
articles.show(100, truncate=False)

In [ ]:
from pyspark.sql.functions import split, lower
articles = articles.select(lower(articles.article).alias('article'))
df_sentences = articles.select(split(articles.article, '[,\.;]').alias('sentences'))

In [ ]:
df_sentences.show(4)

In [ ]:
from pyspark.sql.types import *
def filter_4_plus_wds(sentences):
    if len(sentences.split()) > 3:
        return(True)
def filter_bcl2(sentences):
    if 'bcl2' in sentences:
        return(True)
def bcl2_sentences(sentences):
    sentences = filter(filter_less_3_wds, sentences)
    sentences = filter(filter_bcl2, sentences)
    return(sentences)

bcl2_sentences_udf = udf(bcl2_sentences, ArrayType(StringType()))

In [ ]:
bcl2_df = df_sentences.select(bcl2_sentences_udf(df_sentences.sentences).alias('sentences'))

In [ ]:
bcl2_df.show(5, truncate=False)

In [ ]:
bcl2_df.count()

In [ ]:
articles.count()

In [ ]:
article_urls.count()